In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
!pip install torch_geometric

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.1/63.1 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 51.3 MB/s eta 0:00:00


In [ ]:
import os
import math
import datetime
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
from torch_geometric.nn import GCNConv, global_mean_pool
import torch
import torch.nn as nn
from torch.utils.tensorboard import SummaryWriter
from torch.utils.data import Dataset, random_split
from torch_geometric.loader import DataLoader
from torch_geometric.data import Data
from torch_geometric.utils import dense_to_sparse

In [ ]:
directory = '/content/drive/MyDrive/Thesis'
data_dir = directory + "/Data"
models_dir = directory + "/models"

In [ ]:
distance = pd.read_csv(f'{data_dir}/distance_matrix_centroids.csv')

In [ ]:
distance['end_station_cluster'].min()

74

In [ ]:
timestamps = np.load(f'{data_dir}/demand_graph_timestamps.pkl.npz')
stamps = [timestamps[f'arr_{i}'] for i in range(4379, len(timestamps))] #17516
datetimes = [datetime.datetime(int(arr[0][0]), int(arr[0][1]), int(arr[0][2]), int(arr[0][3])) for arr in stamps]

In [ ]:
datetimes[-1]

datetime.datetime(2024, 3, 31, 23, 0)

In [ ]:
datetimes[0]

datetime.datetime(2022, 7, 2, 13, 0)

In [ ]:
station_clusters = [
    74,  75,  76,  77,  78,  79,  80,  81,  82,  83,  84,  85,  86,
    87,  88,  89,  90,  91,  92,  93,  94,  95,  96,  97,  98,  99,
    100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112,
    113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125,
    126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138,
    139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151,
    152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164,
    165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177,
    178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190,
    191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203,
    205, 206, 208, 209, 210, 211, 212, 213, 214, 216, 217, 218, 219,
    220, 222, 223, 224, 225, 226, 227, 228, 230, 231, 232, 233, 234,
    235, 237, 238, 240, 241, 242, 243, 244, 245, 246, 247, 248, 249,
    250, 251, 252, 253, 255, 257, 258, 261, 264, 265, 266, 274, 275,
    278
]

num_stations = len(station_clusters)

In [ ]:
df = pd.read_csv(f'{data_dir}/final_model_input_partial_scale_2.csv')[[
    'start_station_cluster',
    'started_at_hourly',
    'demand'
  ]]


# Rename the column just once
df = df.rename(columns={"started_at_hourly": "datetime"})
df['datetime'] = pd.to_datetime(df['datetime'])
# Create a DataFrame with all combinations of datetimes and station_clusters
all_combinations = pd.DataFrame(
    [(dt, sc) for dt in datetimes for sc in station_clusters],
    columns=["datetime", "start_station_cluster"]
)

weather_data = pd.read_csv(f"{data_dir}/weather_data.csv")
weather_data['datetime'] = pd.to_datetime(weather_data['started_at_hourly'])

weather_data = weather_data[[
    'datetime',
    'temp',
    'dwpt',
    'rhum',
    'prcp',
    'wdir',
    'wspd',
    'pres'
]]

# Merge with the original DataFrame to include 'demand' where it exists
output_df = all_combinations.merge(
    df[['demand', 'start_station_cluster', 'datetime']],
    on=["datetime", "start_station_cluster"],
    how="left"
)

# Fill missing values with 0
output_df['demand'] = output_df['demand'].fillna(0)

weather_data = weather_data[weather_data['datetime'] >= datetimes[0]]
output_df = output_df[output_df['datetime'] >= datetimes[0]]
target_array = output_df['demand'].values



In [ ]:
del df
del all_combinations
del timestamps
del output_df

In [ ]:
datetimes[-1]

datetime.datetime(2024, 3, 31, 23, 0)

In [ ]:
target = target_array.reshape(-1, num_stations)

In [ ]:
target.max()

355.0

In [ ]:
datetimes[4379]

datetime.datetime(2023, 1, 1, 0, 0)

In [ ]:
stamps[4379][0]

array([2.023e+03, 1.000e+00, 1.000e+00, 0.000e+00, 1.000e+00])

In [ ]:
target.shape

(15321, 183)

In [ ]:
loaded = np.load(f'{data_dir}/demand_graphs.pkl.npz')
demand_graphs = [loaded[f'arr_{i}'] for i in range(4376, 17515)] #4378

In [ ]:
adj_matrices = torch.tensor(demand_graphs)

# Number of stations
num_stations = adj_matrices.shape[1]

<ipython-input-20-4adbaaf13251>:1: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:278.)
  adj_matrices = torch.tensor(demand_graphs)


In [ ]:
del demand_graphs

In [ ]:
len(target)

15321

In [ ]:
from torch_geometric.data import Data
from torch_geometric.utils import dense_to_sparse

# Convert adjacency matrices to PyTorch Geometric format for each hour
def create_datalist(adj_matrices, target, datetimes, weather, target_index=0):
  data_list = []
  for i in range(adj_matrices.shape[0]):  # Iterate over each hour
      adj_matrix = adj_matrices[i]
      y = target[i+target_index]
      datetime_features = datetimes[i+target_index][0]
      weather_features = torch.tensor(weather_data[[
          'dwpt',
          'rhum',
          'prcp',
          'wdir',
          'wspd',
          'pres']].iloc[i+target_index].to_numpy(), dtype=torch.float32)
      # Convert dense adjacency matrix to edge_index (sparse representation)
      edge_index, edge_attr = dense_to_sparse(adj_matrix)
      # Use an identity matrix for node features (can be replaced with other features)
      x = torch.eye(num_stations)
      # Create a Data object
      y = torch.tensor(y, dtype=torch.float32)
      datetime_features = torch.tensor(datetime_features, dtype=torch.float32)
      data = Data(
          x=adj_matrix.float(),
          y=y,
          edge_index=edge_index,
          edge_attr=edge_attr,
          datetime_features=datetime_features,
          weather_features=weather_features
        )
      data_list.append(data)

  print(data_list[0])

  for data in data_list:
      data.edge_attr = data.edge_attr.float()

  return data_list
data_list = create_datalist(adj_matrices, target, stamps, weather_data)

Data(x=[183, 183], edge_index=[2, 195], edge_attr=[195], y=[183], datetime_features=[5], weather_features=[6])


In [ ]:
weather_len = len(['dwpt', 'rhum', 'prcp', 'wdir', 'wspd', 'pres'])

In [ ]:
#del target
del adj_matrices
del target_array

In [ ]:
def create_lagged_dataset(graph_data_list, lag=2):
    lagged_dataset = []
    for i in range(lag, len(graph_data_list)):
        current_graph = graph_data_list[i]
        lag_graphs = graph_data_list[i - lag:i]  # Collect lag graphs
        lagged_dataset.append((current_graph, lag_graphs))
    return lagged_dataset

In [ ]:
lagged_dataset = create_lagged_dataset(data_list)

In [ ]:
len(lagged_dataset)

13137

In [ ]:
len(data_list)

13139

In [ ]:
#data_list[360].to_dict()['edge_attr']

In [ ]:
from torch_geometric.nn import GCNConv
import torch.nn.functional as F

class GNNLayer(torch.nn.Module):
    def __init__(self, in_channels, h1, out_channels, datetime_feats_len, weather_feats_len, num_layers, dropout_prob):
        super(GNNLayer, self).__init__()
        self.convs = nn.ModuleList()
        self.convs.append(GCNConv(in_channels, h1))

        for _ in range(num_layers - 2):
            self.convs.append(GCNConv(h1, h1))
        self.leaky_relu = nn.LeakyReLU()
        self.dropout_prob = dropout_prob
        self.datetime_feats_len = datetime_feats_len
        self.weather_feats_len = weather_feats_len

        self.convs.append(GCNConv(h1, out_channels))

    def forward(self, x, edge_index, batch, datetime_features, weather_features):
      for conv in self.convs[:-1]:
        x = self.leaky_relu(conv(x, edge_index))
        x = F.dropout(x, p=self.dropout_prob, training=self.training)
      x = self.convs[-1](x, edge_index)

      x = global_mean_pool(x, batch)
      datetime_feats = datetime_features[0:self.datetime_feats_len]
      datetime_feats = datetime_feats.unsqueeze(0).expand(x.shape[0], -1)

      weather_feats = weather_features[0:self.weather_feats_len]
      weather_feats = weather_feats.unsqueeze(0).expand(x.shape[0], -1)

      combined_features = torch.cat([x, datetime_feats], dim=1)
      combined_features = torch.cat([combined_features, weather_feats], dim=1)
      return combined_features

In [ ]:
# from torch_geometric.nn import GCNConv
# import torch.nn.functional as F

# class GNNLayer(torch.nn.Module):
#     def __init__(self, in_channels, h1, out_channels, datetime_feats_len, num_layers, dropout_prob):
#         super(GNNLayer, self).__init__()
#         self.convs = nn.ModuleList()
#         self.convs.append(GCNConv(in_channels, h1))

#         for _ in range(num_layers - 2):
#             self.convs.append(GCNConv(h1, h1))
#         self.leaky_relu = nn.LeakyReLU()
#         self.dropout_prob = dropout_prob
#         self.datetime_feats_len = datetime_feats_len
#         self.convs.append(GCNConv(h1, out_channels))

#     def forward(self, x, edge_index, batch, datetime_features):
#       for conv in self.convs[:-1]:
#         x = self.leaky_relu(conv(x, edge_index))
#         x = F.dropout(x, p=self.dropout_prob, training=self.training)
#       x = self.convs[-1](x, edge_index)

#       print(x.shape)
#       x = global_mean_pool(x, batch)
#       datetime_feats = datetime_features[0:self.datetime_feats_len]
#       datetime_feats = datetime_feats.unsqueeze(0).expand(x.shape[0], -1)
#       combined_features = torch.cat([x, datetime_feats], dim=1)
#       print(combined_features.shape)
#       return combined_features

In [ ]:


class GNNForDemandPrediction(torch.nn.Module):
    def __init__(
          self, in_channels, out_channels=50, datetime_feats_len=5, weather_feats_len=6, lag=3, h1=100, num_layers=2, num_layers_lstm=1, fc_hidden_dim = 256, LSTM_hidden_dim=128, dropout_prob=0.2
        ):
        super(GNNForDemandPrediction, self).__init__()

        self.gnn = GNNLayer(in_channels, h1, out_channels, datetime_feats_len, weather_feats_len, num_layers, dropout_prob)

        self.lstm = nn.LSTM(
            (out_channels+datetime_feats_len+weather_feats_len)*lag,
            LSTM_hidden_dim,
            num_layers_lstm,
            batch_first=True,
            bidirectional=True,
            dropout=dropout_prob
        )
        self.fc1 = torch.nn.Linear(LSTM_hidden_dim, fc_hidden_dim)
        self.fc2 = torch.nn.Linear(fc_hidden_dim, in_channels)
        self.dropout_prob = dropout_prob
        self.leaky_relu = nn.LeakyReLU()

    #x, edge_index, batch, datetime_features, weather_features
    def forward(self, batch):
        current_graphs, lag_graphs = batch

        current_graphs.to(device)
        current_x = torch.cat([
            self.gnn(g.x, g.edge_index, g.batch, g.datetime_features, g.weather_features) for g in [current_graphs]
        ], dim=0)
        lag_x_list = []
        for i in lag_graphs:
          #print(i)
          i.to(device)
          x, edge_index, batch_indices, datetime_features, weather_features =\
              i.x, i.edge_index, i.batch, i.datetime_features, i.weather_features
          lag_x = torch.cat([self.gnn(x, edge_index, batch_indices, datetime_features, weather_features)], dim=1)
          #gnn_embedding = gnn_embedding.unsqueeze(1)
          lag_x_list.append(lag_x)
        lag_x = torch.stack(lag_x_list)
        #gnn_embedding = lag_x.mean(dim=0)
        gnn_embedding = current_x
        for i in lag_x:
          gnn_embedding = torch.cat([gnn_embedding, i], dim=1)
        gnn_embedding = gnn_embedding.unsqueeze(1)
        lstm_out, (hidden, _) = self.lstm(gnn_embedding)

        final_hidden_state = lstm_out[:, -1, :]

        x = self.leaky_relu(self.fc1(hidden[-1]))

        x = F.dropout(x, p=self.dropout_prob, training=self.training)

        predicted_demand = self.fc2(x)

        return predicted_demand



In [ ]:


# class GNNForDemandPrediction(torch.nn.Module):
#     def __init__(self, in_channels, out_channels=50, datetime_feats_len=5, h1=100, num_layers=2, num_layers_lstm=1, fc_hidden_dim = 256, LSTM_hidden_dim=128, dropout_prob=0.2):
#         super(GNNForDemandPrediction, self).__init__()

#         self.gnn = GNNLayer(in_channels, h1, out_channels, datetime_feats_len, num_layers, dropout_prob)

#         self.lstm = nn.LSTM(
#             out_channels+datetime_feats_len,
#             LSTM_hidden_dim,
#             num_layers_lstm,
#             batch_first=True,
#             bidirectional=True,
#             dropout=dropout_prob
#         )
#         self.fc1 = torch.nn.Linear(LSTM_hidden_dim, fc_hidden_dim)
#         self.fc2 = torch.nn.Linear(fc_hidden_dim, in_channels)
#         self.dropout_prob = dropout_prob
#         self.leaky_relu = nn.LeakyReLU()

#     def forward(self, x, edge_index, batch, datetime_features):
#         x = self.gnn(x, edge_index, batch, datetime_features)
#         print(x.shape)
#         lstm_out, (hidden, _) = self.lstm(x)
#         print(hidden[-1].shape)
#         x = self.leaky_relu(self.fc1(hidden[-1]))
#         # Predict demand using a fully connected layer
#         predicted_demand = self.fc2(x)  # Shape: [num_edges]
#         print(predicted_demand.shape)
#         #print(predicted_demand.shape)
#         return predicted_demand  # Return predicted demand for each edge
#     # New method to extract node embeddings

#     def get_node_embeddings(self, x, edge_index, batch):
#         x = self.gnn(x, edge_index, batch)
#         return x

In [ ]:
stamps[0][0].shape[0]

5

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


In [ ]:
# Instantiate the model
model = GNNForDemandPrediction(
    in_channels=num_stations,
    h1=100,
    datetime_feats_len = stamps[0][0].shape[0],
    weather_feats_len=weather_len,
    num_layers_lstm=20,
    num_layers=20
  ).to(device)

# Define the loss function (MSE for regression) and the optimizer
criterion = torch.nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=0.001)

In [ ]:
model

GNNForDemandPrediction(
  (gnn): GNNLayer(
    (convs): ModuleList(
      (0): GCNConv(183, 100)
      (1-18): 18 x GCNConv(100, 100)
      (19): GCNConv(100, 50)
    )
    (leaky_relu): LeakyReLU(negative_slope=0.01)
  )
  (lstm): LSTM(183, 128, num_layers=20, batch_first=True, dropout=0.2, bidirectional=True)
  (fc1): Linear(in_features=128, out_features=256, bias=True)
  (fc2): Linear(in_features=256, out_features=183, bias=True)
  (leaky_relu): LeakyReLU(negative_slope=0.01)
)

In [ ]:
num_stations

183

In [ ]:

#dataloader = DataLoader(data_list, batch_size=32, shuffle=True)
dataloader = DataLoader(lagged_dataset, batch_size=32, shuffle=True)

In [ ]:
data_list[0]

Data(x=[183, 183], edge_index=[2, 195], edge_attr=[195], y=[183], datetime_features=[5], weather_features=[6])

In [ ]:
# Training function
def train(model, data_list, optimizer, criterion, epochs=20):
    model.train()  # Set the model to training mode
    for epoch in range(epochs):
        total_loss = 0  # Keep track of total loss

        # Loop over the data (for each hour)
        for batch in dataloader:
            optimizer.zero_grad()  # Clear the gradients
            # Forward pass: predict demand for this hour
            #x, edge_index, batch_indices = batch.x, batch.edge_index, batch.batch  # Edge indices
            predicted_demand = model(batch)#, batch.datetime_features
            # Calculate loss (difference between predicted and actual demand)
            loss = criterion(predicted_demand.reshape(-1), batch[0].y.to(device))

            # Backpropagation and optimization step
            loss.backward()
            optimizer.step()

            total_loss += loss.item()

        if epoch % 10 == 0:
            print(f'Epoch {epoch}, Loss: {total_loss / len(data_list)}')

# Train the model using data from all hours
train(model, data_list, optimizer, criterion, epochs=30)

Epoch 0, Loss: 0.120193269220296
Epoch 10, Loss: 0.11886880121797956
Epoch 20, Loss: 0.11881785755134244


In [ ]:
torch.Size([5856, 50])
torch.Size([32, 55])
torch.Size([32, 55])
torch.Size([128])
torch.Size([183])

torch.Size([183])

In [ ]:
f"{datetime.datetime.now()}"

'2025-01-01 19:47:18.062552'

In [ ]:
model_name = f'{models_dir}/gnn_cnn_weather_datetime_2023_{datetime.datetime.now()}.pth'

In [ ]:
torch.save(model.state_dict(), model_name)



In [ ]:
del data_list

In [ ]:
model_name = f'{models_dir}/gnn_cnn_weather_datetime_2023_2025-01-01 19:47:18.072075.pth'

In [ ]:
# Instantiate the model
model = GNNForDemandPrediction(
    in_channels=num_stations,
    h1=100,
    datetime_feats_len = stamps[0][0].shape[0],
    weather_feats_len=weather_len,
    num_layers_lstm=20,
    num_layers=20
  ).to(device)

model.load_state_dict(torch.load(model_name))

<ipython-input-62-a40e71660d0f>:11: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(model_name))


<All keys matched successfully>

In [ ]:
model

GNNForDemandPrediction(
  (gnn): GNNLayer(
    (convs): ModuleList(
      (0): GCNConv(183, 100)
      (1-18): 18 x GCNConv(100, 100)
      (19): GCNConv(100, 50)
    )
    (leaky_relu): LeakyReLU(negative_slope=0.01)
  )
  (lstm): LSTM(183, 128, num_layers=20, batch_first=True, dropout=0.2, bidirectional=True)
  (fc1): Linear(in_features=128, out_features=256, bias=True)
  (fc2): Linear(in_features=256, out_features=183, bias=True)
  (leaky_relu): LeakyReLU(negative_slope=0.01)
)

In [ ]:
loaded = np.load(f'{data_dir}/demand_graphs.pkl.npz')
demand_graphs = [loaded[f'arr_{i}'] for i in range(17513, len(loaded)-1)] #17515

In [ ]:
datetimes[13138]

datetime.datetime(2024, 1, 1, 0, 0)

In [ ]:
len(demand_graphs)

2186

In [ ]:
15321-13137

2184

In [ ]:
adj_matrices = torch.tensor(demand_graphs)
test_data_list = create_datalist(adj_matrices, target, stamps, weather_data, target_index=13135)

Data(x=[183, 183], edge_index=[2, 113], edge_attr=[113], y=[183], datetime_features=[5], weather_features=[6])


In [ ]:
lagged_dataset = create_lagged_dataset(test_data_list)

In [ ]:
len(lagged_dataset)

2184

In [ ]:
del demand_graphs
del adj_matrices

In [ ]:
test_dataloader = DataLoader(lagged_dataset, batch_size=1, shuffle=True)

In [ ]:

from sklearn.metrics import mean_absolute_percentage_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
criterion = torch.nn.MSELoss()
# Testing the model (evaluation mode)
def test(model, dataloader):
    model.eval()  # Set the model to evaluation mode
    total_loss = 0

    preds = []
    actual = []
    with torch.no_grad():  # No need to track gradients during testing
        for batch in dataloader:
            predicted_demand = model(batch)  # Forward pass
            #print(predicted_demand.reshape(-1).shape)

            preds.extend(predicted_demand.cpu().reshape(-1))
            #print(batch.y.reshape(-1).shape)
            actual.extend(batch[0].y.cpu().reshape(-1))
            #loss = criterion(predicted_demand, data.y)  # Compute loss
            #total_loss += loss.item()
    df = pd.DataFrame({"actual": actual, "pred": preds})
    non_zero = df.query('actual != 0')
    zeros = df.query('actual == 0')


    mse_score = mean_squared_error(df['actual'], df['pred'])
    rmse_score = np.sqrt(mse_score)
    mae_score = mean_absolute_error(df['actual'], df['pred'])
    mape_score = mean_absolute_percentage_error(df['actual']+1, df['pred']+1)
    print("overall")
    print("MSE:", mse_score)
    print("RMSE:", rmse_score)
    print("MAE:", mae_score)
    print("MAPE:", mape_score)


    mse_score = mean_squared_error(non_zero['actual'], non_zero['pred'])
    rmse_score = np.sqrt(mse_score)
    mae_score = mean_absolute_error(non_zero['actual'], non_zero['pred'])
    mape_score = mean_absolute_percentage_error(non_zero['actual']+1, non_zero['pred']+1)
    print()
    print("Non-zero")
    print("MSE:", mse_score)
    print("RMSE:", rmse_score)
    print("MAE:", mae_score)
    print("MAPE:", mape_score)

    mse_score = mean_squared_error(zeros['actual'], zeros['pred'])
    rmse_score = np.sqrt(mse_score)
    mae_score = mean_absolute_error(zeros['actual'], zeros['pred'])
    mape_score = mean_absolute_percentage_error(zeros['actual']+1, zeros['pred']+1)
    print()
    print("Zeros")
    print("MSE:", mse_score)
    print("RMSE:", rmse_score)
    print("MAE:", mae_score)
    print("MAPE:", mape_score)
    #print(f'Test Loss: {total_loss / len(data_list)}')

# Test the model
test(model, test_dataloader)

overall
MSE: 5.266619634331806
RMSE: 2.2949116833402994
MAE: 0.7660327558780519
MAPE: 0.30527436352371845

Non-zero
MSE: 22.868664499622163
RMSE: 4.782119247741754
MAE: 2.6430262334403305
MAPE: 0.4484125488171445

Zeros
MSE: 0.5890147791046572
RMSE: 0.7674729826545409
MAE: 0.26723651538056054
MAPE: 0.2672365176017819


In [ ]:
#datetime features
2.351703

#datetime + weather features
2.3364873

#datetime + weather features + lag
2.3000178

#datetime + weather features + lag (20 layers)
2.2951133

In [ ]:
model

GNNForDemandPrediction(
  (conv1): GCNConv(183, 100)
  (conv2): GCNConv(100, 50)
  (fc): Linear(in_features=50, out_features=1, bias=True)
)

In [ ]:
model.get_node_embeddings(data_list[i])[0].shape

torch.Size([50])

In [ ]:
model.get_node_embeddings(data_list[i])

tensor([[-0.0548, -0.0327, -0.0491,  ..., -0.0474,  0.0586, -0.0672],
        [-0.0548, -0.0327, -0.0491,  ..., -0.0474,  0.0586, -0.0672],
        [-0.0548, -0.0327, -0.0491,  ..., -0.0474,  0.0586, -0.0672],
        ...,
        [-0.0548, -0.0327, -0.0491,  ..., -0.0474,  0.0586, -0.0672],
        [-0.0548, -0.0327, -0.0491,  ..., -0.0474,  0.0586, -0.0672],
        [-0.0548, -0.0327, -0.0491,  ..., -0.0474,  0.0586, -0.0672]],
       grad_fn=<AddBackward0>)

In [ ]:
torch.save(model.state_dict(), f'{data_dir}/gnn_cnn4.pth')

In [ ]:
model = GNNForDemandPrediction(
    in_channels=num_stations,
    h1=100,
    h2=80,
    h3=60,
    out_channels=50
)

In [ ]:
model.load_state_dict(torch.load(f'{data_dir}/gnn_cnn4.pth'))

<ipython-input-37-41fcb6f99048>:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(f'{data_dir}/gnn_cnn4.pth'))


<All keys matched successfully>

In [ ]:
data_list[i].to_dict()['edge_index']

tensor([[  3,   3,  19,  21,  22,  23,  28,  28,  28,  28,  28,  30,  30,  30,
          38,  38,  38,  40,  40,  40,  40,  40,  40,  40,  45,  45,  50,  50,
          53,  54,  54,  54,  59,  61,  61,  61,  61,  61,  61,  62,  62,  62,
          62,  64,  64,  64,  64,  64,  64,  66,  66,  66,  66,  68,  71,  71,
          71,  71,  77,  77,  77,  77,  77,  77,  77,  80,  80,  81,  81,  81,
          81,  81,  81,  81,  81,  81,  83,  83,  83,  83,  83,  83,  83,  83,
          83,  83,  83,  83,  83,  84,  84,  84,  86,  86,  87,  87,  87,  87,
          87,  88,  88,  88,  88,  88,  88,  88,  88,  90,  90,  90,  90,  90,
          90,  90,  90,  90,  90,  93,  93,  94,  94,  98,  99, 100, 100, 100,
         100, 101, 101, 101, 103, 103, 103, 103, 103, 104, 104, 104, 104, 104,
         104, 104, 104, 104, 105, 105, 108, 108, 108, 108, 108, 108, 108, 108,
         108, 108, 119, 119, 120, 120, 120, 120, 120, 120, 120, 120, 122, 122,
         122, 122, 122, 122, 125, 127, 127, 130, 131

In [ ]:
for i in range(2000):
  print(len(data_list[i].to_dict()['edge_attr']))

57
136
116
30
13
11
18
20
36
55
88
135
161
209
212
235
212
110
92
55
53
46
35
41
24
16
13
10
11
9
11
26
55
87
153
139
198
220
208
191
168
148
120
71
46
66
61
27
20
12
10
2
10
21
54
105
156
136
140
146
164
192
156
153
121
120
91
61
30
36
18
13
7
6
4
2
11
18
39
93
136
122
103
128
135
123
144
156
237
271
223
133
86
73
45
24
19
11
9
9
12
24
64
125
169
156
123
143
190
158
163
213
254
269
233
176
107
77
54
42
16
8
11
3
9
18
69
143
218
188
136
154
185
158
179
204
252
276
229
169
112
76
48
29
12
8
3
12
10
21
51
96
133
107
59
91
110
128
165
200
285
300
234
152
114
87
80
55
46
26
28
5
13
8
21
40
88
139
217
250
290
291
318
271
296
216
136
138
102
66
53
55
44
36
19
19
8
9
22
37
68
114
177
261
280
256
321
280
308
226
124
103
100
65
52
38
19
11
10
11
11
21
62
156
240
191
127
142
181
176
172
203
281
317
245
166
118
87
49
23
16
9
7
4
7
18
72
175
272
169
148
158
192
173
184
258
298
355
292
222
146
94
72
47
30
16
12
5
8
22
70
177
261
204
171
172
219
191
234
229
289
359
270
221
153
120
73
37
25
18
10
3
9

In [ ]:
model.eval()

GNNForDemandPrediction(
  (conv1): GCNConv(188, 100)
  (conv2): GCNConv(100, 80)
  (conv3): GCNConv(80, 60)
  (conv4): GCNConv(60, 50)
  (fc): Linear(in_features=50, out_features=1, bias=True)
)

In [ ]:
loaded = np.load(f'{data_dir}/demand_graphs.pkl.npz')
demand_graphs = [loaded[f'arr_{i}'] for i in range(8760, 9760)]
adj_matrices = torch.tensor(demand_graphs)

from torch_geometric.data import Data
from torch_geometric.utils import dense_to_sparse

# Convert adjacency matrices to PyTorch Geometric format for each hour
data_list_input = []

for i in range(adj_matrices.shape[0]):  # Iterate over each hour
    adj_matrix = adj_matrices[i]

    # Convert dense adjacency matrix to edge_index (sparse representation)
    edge_index, edge_attr = dense_to_sparse(adj_matrix)

    # Use an identity matrix for node features (can be replaced with other features)
    x = torch.eye(num_stations)

    # Create a Data object
    data = Data(x=x, edge_index=edge_index, edge_attr=edge_attr)
    data_list_input.append(data)

print(data_list_input[0])

for data in data_list_input:
    data.edge_attr = data.edge_attr.float()

Data(x=[188, 188], edge_index=[2, 179], edge_attr=[179])


In [ ]:
for data in data_list_input:
  m = model.get_node_embeddings(data)
  print(m[6])

  print()
  #[6]

#.shape

Streaming output truncated to the last 5000 lines.
         4.1210e-02,  2.2405e-02, -7.6899e-02,  2.9430e-02,  6.1244e-02,
         2.9741e-02, -1.9238e-02,  8.3620e-02, -2.8631e-02, -5.4592e-02,
         2.2581e-02, -7.1912e-02,  5.7599e-02, -5.5164e-02, -5.7451e-02,
        -1.3684e-01, -2.7002e-02,  3.9655e-02,  6.9899e-02,  2.6385e-07,
        -9.0836e-02, -7.6720e-02, -4.2684e-02, -8.8408e-02,  7.4278e-02,
         2.5705e-01, -7.2568e-02,  9.4929e-02,  5.3460e-02,  5.6016e-02],
       grad_fn=<SelectBackward0>)

tensor([ 6.5043e-02,  6.1028e-02, -6.4026e-02,  8.6542e-02, -3.0812e-02,
         3.1783e-02,  1.0259e-05,  6.3434e-02,  5.0898e-02,  4.5800e-02,
         4.4471e-02, -7.3395e-02,  8.5991e-02,  9.7119e-02, -3.4149e-02,
         7.5424e-02,  3.9552e-02, -3.2949e-02, -8.5411e-02,  4.3779e-02,
         4.1210e-02,  2.2405e-02, -7.6899e-02,  2.9430e-02,  6.1244e-02,
         2.9741e-02, -1.9238e-02,  8.3620e-02, -2.8631e-02, -5.4592e-02,
         2.2581e-02, -7.1912e-02,  5.

KeyboardInterrupt: 

In [ ]:
data_list[300].to_dict()['edge_attr']

tensor([1., 1., 1., 1., 1., 1., 2., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 2., 1., 1., 2., 2., 3., 1., 1., 1., 1., 1., 1., 2., 3.,
        1., 1., 2., 1., 1., 1., 2., 1., 1., 1., 1., 1., 1., 1., 2., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 2., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 2., 1., 1., 1., 1., 1., 1., 1.,
        2., 1., 1., 1., 1., 1., 2., 1., 1., 1., 1., 1., 1., 1., 1., 2., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 2., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1.])

In [ ]:
model(data_list[12])

tensor([1.2584, 1.4187, 1.2692, 1.2386, 1.2737, 1.4849, 1.5614, 1.5522, 1.6454,
        1.4131, 1.4009, 1.2386, 1.2964, 1.2584, 1.2304, 1.2622, 1.2304, 1.2304,
        1.4105, 1.4642, 1.5222, 1.3701, 1.2386, 1.3000, 1.2963, 1.3805, 1.3281,
        1.2996, 1.2661, 1.2529, 1.2940, 1.2642, 1.4775, 1.6878, 1.5028, 1.5853,
        1.5329, 1.6433, 1.4820, 1.5067, 1.3422, 1.2612, 1.2584, 1.4187, 1.3925,
        1.5853, 1.3422, 1.5408, 1.3720, 1.4042, 1.3511, 1.3977, 1.5011, 1.3958,
        1.3607, 1.2798, 1.3677, 1.4109, 1.3779, 1.3494, 1.3315, 1.3193, 1.3494,
        1.2938, 1.4363, 1.4520, 1.4632, 1.3677, 1.2584, 1.3595, 1.5923, 1.4898,
        1.5385, 1.3753, 1.4290, 1.5399, 1.4027, 1.5222, 1.4413, 1.5408, 1.4599,
        1.5988, 1.5978, 1.4622, 1.4771, 1.4967, 1.4002, 1.3795, 1.2584, 1.4255,
        1.4765, 1.5315, 1.4791, 1.4106, 1.4400, 1.5400, 1.2651, 1.2695, 1.3688,
        1.2703, 1.2397, 1.3534, 1.3790, 1.4208, 1.2612, 1.5517, 1.5604, 1.6423,
        1.6783, 1.6817, 1.5482, 1.5900, 